In [1]:
from spimi import SPIMI
import pandas as pd

[nltk_data] Downloading package punkt to /home/ambar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
# testeo
path = './dataset/'
data_path = path + 'spotify_songs.csv'

data = pd.read_csv(data_path)

In [36]:
data1000 = data.head(1000)
data5000 = data.head(5000)
data10000 = data.head(10000)
data18000 = data

In [37]:
spimi = SPIMI(size_per_block=10240*4,
              path_block= './.temp1000/',
              output_folder='./blocks1000/',
              ram_limit=1024*1024*1024*4,
              size_per_block_out= 1024*4)

spimi.BSBIndexConstuction(data1000)


spimi = SPIMI(size_per_block=10240*4,
              path_block= './.temp5000/',
              output_folder='./blocks5000/',
              ram_limit=1024*1024*1024*4,
              size_per_block_out= 1024*4)

spimi.BSBIndexConstuction(data5000)

spimi = SPIMI(size_per_block=10240*4,
              path_block= './.temp10000/',
              output_folder='./blocks10000/',
              ram_limit=1024*1024*1024*4,
              size_per_block_out= 1024*4)

spimi.BSBIndexConstuction(data10000)

spimi = SPIMI(size_per_block=10240*4,
              path_block= './.temp18000/',
              output_folder='./blocks18000/',
              ram_limit=1024*1024*1024*4,
              size_per_block_out= 1024*4)

spimi.BSBIndexConstuction(data18000)

KeyboardInterrupt: 

In [ ]:
import numpy as np
import os
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import pandas as pd
import bisect

class CosineSimilaritySearch:
    def __init__(self, block_folder, data):
        self.block_folder = block_folder
        self.data = data
        self.num_docs = len(data)
        self.language_map = {
            'es': 'spanish',
            'en': 'english',
            'fr': 'french',
            'de': 'german',
            'it': 'italian'
        }

    def preprocess(self, text, lang='en'):
        lang = self.language_map.get(lang, 'english')
        stop_words = stopwords.words(lang)
        stemmer = SnowballStemmer(language=lang)

        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', text)

        words = []
        for word in text.split():
            if word not in stop_words:
                words.append(stemmer.stem(word))

        return words

    def calculate_query_vector(self, query_terms, df_dict):
        tf_query = defaultdict(int)
        for term in query_terms:
            tf_query[term] += 1

        query_vector = {}
        for term, tf in tf_query.items():
            idf = np.log((self.num_docs / df_dict[term])) if term in df_dict and df_dict[term] > 0 else 0
            query_vector[term] = tf * idf
            # print(f"Término: {term}, TF: {tf}, IDF: {idf}, TF-IDF: {query_vector[term]}")
        return query_vector
    
    def load_block_terms(self, query_terms):
        term_postings = {}
        df_dict = {}

        for filename in os.listdir(self.block_folder):
            with open(os.path.join(self.block_folder, filename), 'r') as file:
                for line in file:
                    term, rest = line.split(" (DF: ")
                    if self.binary_search(query_terms, term):
                        df, postings = rest.split("): ")
                        df = int(df)
                        df_dict[term] = df
                        
                        term_postings[term] = []
                        postings_list = postings.strip().split("), (")
                        for posting in postings_list:
                            doc_id, tf = map(int, posting.strip("()").split(", "))
                            term_postings[term].append((doc_id, tf))
        return term_postings, df_dict

    def binary_search(self, sorted_list, item):
        index = bisect.bisect_left(sorted_list, item)
        return index < len(sorted_list) and sorted_list[index] == item

    def cosine_similarity(self, query_vector, term_postings):
        doc_scores = defaultdict(float)  # Producto punto acumulado para cada documento
        doc_norms = defaultdict(float)  # Acumular las normas de cada documento
        query_norm = np.sqrt(np.sum(np.square(list(query_vector.values()))))  # Norma de la consulta

        # Construcción de vectores de documentos y cálculo del producto punto
        for term, query_weight in query_vector.items():
            if term in term_postings:
                for doc_id, tf in term_postings[term]:
                    tf_weight = tf  # TF del término en el documento
                    doc_scores[doc_id] += query_weight * tf_weight  # Producto punto acumulado
                    doc_norms[doc_id] += tf_weight**2  # Sumar al cuadrado para calcular norma del documento

        # Cálculo de similitud coseno
        for doc_id in doc_scores:
            doc_norm = np.sqrt(doc_norms[doc_id])  # Norma del documento
            if query_norm != 0 and doc_norm != 0:
                doc_scores[doc_id] = doc_scores[doc_id] / (query_norm * doc_norm)  # Similitud coseno
            else:
                doc_scores[doc_id] = 0.0  # Evitar división por cero

        # Ordenar por similitud coseno descendente
        return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        
    def get_top_k_similar_documents(self, query, lang='en', k=5):
        query_terms = self.preprocess(query, lang=lang)
        print(f"Términos de la consulta procesados: {query_terms}")
        term_postings, df_dict = self.load_block_terms(query_terms)
        query_vector = self.calculate_query_vector(query_terms, df_dict)
        
        doc_scores = self.cosine_similarity(query_vector, term_postings)
        doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)[:k]

        results = []
        for doc_id, score in doc_scores:
            try:
                doc_details = self.data.iloc[doc_id].to_dict()
                doc_details["Cosine Similarity Score"] = score
                results.append(doc_details)
            except IndexError:
                print(f"El documento con ID {doc_id} no está disponible en los datos.")

        results_df = pd.DataFrame(results)
        results_df = results_df[["track_name", "track_artist", "lyrics", "Cosine Similarity Score"]]
        results_df.columns = ["Song Title", "Artist", "Lyrics", "Similarity Score"]
        results_df.index = range(1, len(results) + 1)
        return results_df

    def search(self, query, lang='en'):
        query_terms = self.preprocess(query, lang=lang)
        term_postings, df_dict = self.load_block_terms(query_terms)
        query_vector = self.calculate_query_vector(query_terms, df_dict)
        return self.cosine_similarity(query_vector, term_postings)


In [45]:


block_folder = './blocks1/'
search_engine = CosineSimilaritySearch(block_folder, data)
results_df = search_engine.get_top_k_similar_documents("mayor que yo", lang='es', k=5)
print("Top K documentos más similares:")
print(results_df)

Términos de la consulta procesados: ['mayor']
Top K documentos más similares:
       Song Title         Artist  \
1     Die To Live        Volbeat   
2  Mayor Que Yo 3     Luny Tunes   
3       Civil War  Guns N' Roses   

                                              Lyrics  Similarity Score  
1  It's 7:02, breaking all the rules Dance the bo...               1.0  
2  Letra de "Mayor Que Yo 3" ft. Don Omar, Wisin,...               1.0  
3  What we've got here is failure to communicate ...               1.0  


In [58]:

block_folder = './blocks10000/'
search_engine = CosineSimilaritySearch(block_folder, data10000)
results_df = search_engine.get_top_k_similar_documents("mayor que yo", lang='es', k=50)
print("Top K documentos más similares:")
print(results_df)

Términos de la consulta procesados: ['mayor']
Top K documentos más similares:
                                           Song Title                Artist  \
1                                         Die To Live               Volbeat   
2                                      Mayor Que Yo 3            Luny Tunes   
3                                           Civil War         Guns N' Roses   
4                                 Rebelde por Defecto              ToteKing   
5                                          Quiero Mas                 Ozuna   
6                                                  Ho              Ludacris   
7                                       Ghetto Cowboy              Mo Thugs   
8                                              Diavla             Chris Viz   
9                                          King Kunta        Kendrick Lamar   
10                                            Ferrari                  Duki   
11                                  Boogie Down Bronx